In [1]:
from compute import Var, eqvar
from inputresolver import reassigneq, eqvars, eqsonly, default_out, mdf_order, default_in, idx_eqlist
from inputresolver import getallvars, invert_edges, resolve, reassign
from representations import bipartite_repr
from compute_utils import get_outputs, check_eqs, print_out
from pprint import pprint
import networkx as nx
import sympy as sp
import numpy as np
import openmdao.api as om

Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [2]:
from compute import Var, eqvar, ureg, buildidpvars, coupled_run
from compute_utils import get_outputs, print_out
from inputresolver import idx_eqlist, eqvars, default_in, default_out, mdf_order
import sympy as sp
import openmdao.api as om

In [3]:
ureg.define('USD = [currency] ')

## Part 1

## Variables
For which we have values at the time

In [4]:
IFOV = Var('IFOV', 7.272e-5, 'rad') 
r = Var('r', 9.257e19, 'm')
l = Var('lambda', 5e-7, 'm')
d = Var('d', 3e-5, 'm')
Q = Var('Q', 1.1)
rho = Var('rho', 1.22) 
k = Var('k', 1.381e-23, 'J/K')
T = Var('T', 5785, 'K')
h = Var('h', 6.626e-34, 'J*s')
c = Var('c', 2.998e8, 'm/s')
tau = Var('tau', 1)
dl = Var('Delta_lambda', 2e-6, 'm')
R = Var('R', 6.96e8, 'm')
dV = Var('Delta_V', 0, 'm/s')
g = Var('g', 9.8, 'm/s^2')
isp = Var('I_sp', 450, 's')
ct = Var('c_t', 1163, 'USD/kg')
ms = Var('m_s', 1175, 'kg/m^2')
QE = Var('Q_E', 0.5)
Nr = Var('N_r', 25)
tau0 = Var('tau_0', 0.75)
Ti = Var('T_i', 30, 's')
a = Var('alpha', 50e3, 'USD/m^3')
SNR = Var('SNR',)
CT = Var('C_T', 1e6, 'USD', never_output=True)

## Equations

In [5]:
Y, eq1 = eqvar('Y', IFOV*r, 'm')
f, eq2 = eqvar('f', r*d/Y, 'm')
D, eq3 = eqvar('D', 2*rho*l*Q*f/d, 'm')
tr, eq4 = eqvar('theta_r', rho*l/D, 'rad')
mt, eq5 = eqvar('mt', ms*D**2, 'kg')
mi, eq6 = eqvar('m_i', mt*sp.exp(dV/(g*isp)), 'kg')
CD, eq7 = eqvar('C_D', a*D**3, 'USD')
CT, eq8 = eqvar('C_T', CD+ct*mi,'USD')
Hl, eq9 = eqvar('H_lambda', 2*sp.pi*h*c**2/l**5*1/(sp.exp(c*h/(k*T*l))-1), 'W/m^3')
L, eq10 = eqvar('L', 1/4*R**2*Hl*tau*dl, 'W/sr')
Pin, eq11 = eqvar('P_in', sp.pi*(D/(2*r))**2*L, 'W')
Hi, eq12 = eqvar('H_i', Pin*tau0*Ti, 'J')
Np, eq13 = eqvar('N_p', Hi*l/(h*c))
SNR, eq14 = eqvar('SNR', Np*QE/sp.sqrt(Nr**2+Np*QE))
eq_list=[eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13, eq14]

Generate an equation index (a dictionary):

In [6]:
eqs = idx_eqlist(eq_list)

Get all variables of residual form of equation

In [7]:
eqv = eqvars(eqs)

In [8]:
eqv

{0: {IFOV, Y, r},
 1: {Y, d, f, r},
 2: {D, Q, d, f, lambda, rho},
 3: {D, lambda, rho, theta_r},
 4: {D, m_s, mt},
 5: {Delta_V, I_sp, g, m_i, mt},
 6: {C_D, D, alpha},
 7: {C_D, C_T, c_t, m_i},
 8: {H_lambda, T, c, h, k, lambda},
 9: {Delta_lambda, H_lambda, L, R, tau},
 10: {D, L, P_in, r},
 11: {H_i, P_in, T_i, tau_0},
 12: {H_i, N_p, c, h, lambda},
 13: {N_p, N_r, Q_E, SNR}}

Get user defined default outputs and inputs

In [9]:
dout = default_out(eqs)
dins = default_in(eqs)

In [10]:
print(dout)

{0: Y, 1: f, 2: D, 3: theta_r, 4: mt, 5: m_i, 6: C_D, 7: C_T, 8: H_lambda, 9: L, 10: P_in, 11: H_i, 12: N_p, 13: SNR}


Get MDF order

In [11]:
order = mdf_order(eqv, dout)

In [12]:
order

[8, 9, 0, 1, 2, 10, 11, 12, 13, 6, 4, 5, 7, 3]

Run computation with openMDAO

In [13]:
prob = om.Problem()
model = prob.model
buildidpvars(dins, model)
counter = coupled_run(eqs, order, (), model, model, 0)
prob.setup()
prob.run_model()

eq8 H_lambda 2*pi*c**2*h/(lambda**5*(exp(c*h/(T*k*lambda)) - 1)) None
eq9 L 0.25*Delta_lambda*H_lambda*R**2*tau None
eq0 Y IFOV*r None
eq1 f d*r/Y None
eq2 D 2*Q*f*lambda*rho/d None
eq10 P_in pi*D**2*L/(4*r**2) None
eq11 H_i P_in*T_i*tau_0 None
eq12 N_p H_i*lambda/(c*h) None
eq13 SNR N_p*Q_E/sqrt(N_p*Q_E + N_r**2) None
eq6 C_D D**3*alpha None
eq4 mt D**2*m_s None
eq5 m_i mt*exp(Delta_V/(I_sp*g)) None
eq7 C_T C_D + c_t*m_i None
eq3 theta_r lambda*rho/D None


Print the output

In [14]:
out = get_outputs(eqs, model)
print_out(out, False)

{d: '3e-05',
 lambda: '5e-07',
 Y: '6.732e+15',
 alpha: '5e+04',
 Delta_V: '0.22',
 IFOV: '7.272e-05',
 c_t: '1163',
 h: '6.626e-34',
 k: '1.381e-23',
 tau: '1',
 H_i: '1.42e-17',
 T_i: '30',
 f: '0.41',
 c: '2.998e+08',
 P_in: '6.31e-19',
 N_p: '35.74',
 r: '9.257e+19',
 g: '9.80',
 Q_E: '0.50',
 m_i: '0.40',
 N_r: '25',
 mt: '0.40',
 T: '5785',
 theta_r: '3.305e-05',
 Q: '1.10',
 H_lambda: '8.347e+13',
 R: '6.96e+08',
 C_D: '0.31',
 tau_0: '0.75',
 I_sp: '450',
 C_T: '465.73',
 L: '2.022e+25',
 m_s: '1175',
 D: '0.02',
 SNR: '0.70',
 Delta_lambda: '2e-06',
 rho: '1.22'}

# Part 2

In [15]:
from representations import bipartite_repr
from inputresolver import getallvars, invert_edges, reassign
import networkx as nx
from functools import reduce

In [16]:
force_input = [SNR, CT]
not_input = [IFOV, Ti] #allow to vary, meaning output
#force_input = [mi]
#not_input = [l]

In [17]:
SNR.varval = 10
CT.varval = 1e6
#mi.varval = 60e3

We redefine the problem, where we exchange the inputs and ouputs

In [18]:
new_ins = dins.union(force_input) - set(not_input)
new_eqv = {key: {elt for elt in outsetposs if not elt in new_ins}
        for key, outsetposs in eqv.items()}

In [19]:
new_eqv

{0: {IFOV, Y},
 1: {Y, f},
 2: {D, f},
 3: {D, theta_r},
 4: {D, mt},
 5: {m_i, mt},
 6: {C_D, D},
 7: {C_D, m_i},
 8: {H_lambda},
 9: {H_lambda, L},
 10: {D, L, P_in},
 11: {H_i, P_in, T_i},
 12: {H_i, N_p},
 13: {N_p}}

Setup for matching

In [20]:
G, edges_original = bipartite_repr(new_eqv)

In [21]:
M = nx.bipartite.maximum_matching(G)

In [22]:
d = {key:M[key] for key in dout.keys()}

In [23]:
print(d)

{0: IFOV, 1: Y, 2: f, 3: theta_r, 4: mt, 5: m_i, 6: D, 7: C_D, 8: H_lambda, 9: L, 10: P_in, 11: T_i, 12: H_i, 13: N_p}


In [24]:
new_eqs=reassign(eqs, d)
new_eqv = eqvars(new_eqs)
new_order = mdf_order(new_eqv, d)
new_ins = reduce(set.union, new_eqv.values()) - set(d.values())

In [25]:
new_eqs

{0: (IFOV, Y/r),
 1: (Y, d*r/f),
 2: (f, D*d/(2*Q*lambda*rho)),
 3: (theta_r, lambda*rho/D),
 4: (mt, D**2*m_s),
 5: (m_i, mt*exp(Delta_V/(I_sp*g))),
 6: (D, (C_D/alpha)**(1/3)),
 7: (C_D, C_T - c_t*m_i),
 8: (H_lambda, 2*pi*c**2*h/(lambda**5*(exp(c*h/(T*k*lambda)) - 1))),
 9: (L, 0.25*Delta_lambda*H_lambda*R**2*tau),
 10: (P_in, pi*D**2*L/(4*r**2)),
 11: (T_i, H_i/(P_in*tau_0)),
 12: (H_i, N_p*c*h/lambda),
 13: (N_p, SNR*(SNR + sqrt(4*N_r**2 + SNR**2))/(2*Q_E))}

In [26]:
new_order

[13, (7, 5, 4, 6), 12, 8, 9, 3, 2, 10, 11, 1, 0]

In [27]:
prob = om.Problem()
model = prob.model
buildidpvars(new_ins, model)
counter = coupled_run(new_eqs, new_order, (), model, model)
prob.setup()
prob.run_model()

eq13 N_p SNR*(SNR + sqrt(4*N_r**2 + SNR**2))/(2*Q_E) None
eq7 C_D C_T - c_t*m_i None
eq5 m_i mt*exp(Delta_V/(I_sp*g)) None
eq4 mt D**2*m_s None
eq6 D (C_D/alpha)**(1/3) None
eq12 H_i N_p*c*h/lambda None
eq8 H_lambda 2*pi*c**2*h/(lambda**5*(exp(c*h/(T*k*lambda)) - 1)) None
eq9 L 0.25*Delta_lambda*H_lambda*R**2*tau None
eq3 theta_r lambda*rho/D None
eq2 f D*d/(2*Q*lambda*rho) None
eq10 P_in pi*D**2*L/(4*r**2) None
eq11 T_i H_i/(P_in*tau_0) None
eq1 Y d*r/f None
eq0 IFOV Y/r None

group1.group2
NL: Newton Converged in 6 iterations


In [28]:
out = get_outputs(eqs, model)
print_out(out, False)

{d: '3e-05',
 lambda: '5e-07',
 Y: '1.474e+14',
 alpha: '5e+04',
 Delta_V: '0.22',
 IFOV: '1.593e-06',
 c_t: '1163',
 h: '6.626e-34',
 k: '1.381e-23',
 tau: '1',
 H_i: '2.423e-16',
 T_i: '0.25',
 f: '18.83',
 c: '2.998e+08',
 P_in: '1.315e-15',
 N_p: '609.90',
 r: '9.257e+19',
 g: '9.80',
 Q_E: '0.50',
 m_i: '834.13',
 N_r: '25',
 mt: '834.09',
 T: '5785',
 theta_r: '7.24e-07',
 Q: '1.10',
 H_lambda: '8.347e+13',
 R: '6.96e+08',
 C_D: '2.99e+04',
 tau_0: '0.75',
 I_sp: '450',
 C_T: '1e+06',
 L: '2.022e+25',
 m_s: '1175',
 D: '0.84',
 SNR: '10',
 Delta_lambda: '2e-06',
 rho: '1.22'}